# LTI認証連携の設定を行う

---

Moodleとの間で LTI 1.3による認証連携を設定する。

## 概要

### LTI連携とは

LTI（Learning Tools Interoperability）は、LMS（Learning Management System）と外部学習ツール間の標準的な連携仕様です。CoursewareHubでは LTI 1.3 を使用してMoodleとの認証連携を実現し、シームレスな学習環境を提供します。

**LTI連携の役割分担:**
- **Platform（Moodle）**: ユーザ認証・認可を管理し、コース情報を提供
- **Tool（CoursewareHub）**: Jupyter Notebook による学習環境を提供

**LTI連携による利用シーン:**
- **講師**: Moodleのコースページから CoursewareHub にアクセスし、Jupyter Notebookの教材を作成・配布
- **受講者**: Moodleのコースページから CoursewareHub にアクセスし、専用のJupyter Notebook環境で演習を実行
- **管理者**: 複数コースの学習環境を一元管理

### CoursewareHubにおけるLTI実装

CoursewareHubのLTI実装では、以下の特徴を提供します：

**認証・認可の流れ:**
<!--
```mermaid
sequenceDiagram
    participant Student as 受講者
    participant Moodle as Moodle<br/>(Platform)
    participant CWH as CoursewareHub<br/>(Tool)
    
    Student->>Moodle: ログイン
    Student->>Moodle: CoursewareHubリンクをクリック
    Moodle->>CWH: LTI Launch Request<br/>(認証情報+コース情報)
    CWH->>CWH: ユーザ認証・環境準備
    CWH->>Student: Notebook環境表示
    Student->>CWH: 演習・課題実行
```
-->

![認証・認可の流れ](images/cw-411-01.png)

**主要機能:**
- **シングルサインオン（SSO）**: Moodleでの認証情報を使用してCoursewareHubに自動ログイン
- **コース識別機能**: LTI連携パラメータによるコース単位での環境分離
- **複数コース対応**: 単一のCoursewareHubインスタンスで複数コースを管理
- **自動環境準備**: ユーザ・コース毎に独立したNotebook環境を自動作成

### 対象ユーザと前提条件

**対象ユーザ:**
- **講師**: 教材作成・配布および受講者管理を行う
- **受講者**: Jupyter Notebookによる演習・課題を実行する

**前提条件:**
- Moodle環境が稼働しており、管理者権限を有すること
- CoursewareHub から Moodle へのアクセスが可能であること

### ユーザマッピング

CoursewareHubとMoodleの間では、**メールアドレス**を主要な識別子としてユーザを対応づけます。

**基本的な仕組み:**
- **Moodle**: 各ユーザアカウントに登録されているメールアドレス
- **CoursewareHub**: LTI認証時に受信したメールアドレスをもとに自動的にユーザアカウントを作成・管理
- **対応関係**: 同一のメールアドレスを持つユーザが同一人物として認識される

**LTI認証時の情報伝達:**
MoodleからCoursewareHubへのLTI Launch時に**メールアドレス**のみが送信されます。

**重要な留意点:**
- 各ユーザは一意のメールアドレスを使用する必要があります
- 学習期間中のメールアドレス変更は避けることを推奨します

### LTI連携設定のおおまかな流れ

LTI連携の設定は以下の5つのステップで構成されます：

**ステップ1: 事前準備・要件確認**
- CoursewareHubとMoodleの疎通確認
- 権限確認、要件整理
- 複数コース運用時の容量・管理体制の検討

**ステップ2: Moodle でのLTI外部ツール登録**
- Moodle のサイト全体にCoursewareHubをLTI外部ツールとして登録
- プラットフォームID、クライアントID、デプロイメントIDの取得

**ステップ3: CoursewareHubでのLTI設定**
- CoursewareHubにLTI連携設定ファイルを配置
- auth-proxyサービスの更新

**ステップ4: Moodleコースでのリンク作成**
- Moodleの各コースにCoursewareHubへのリンクを作成
- 複数コース対応のカスタムパラメータ設定

**ステップ5: 動作確認・運用開始**
- 実際の認証動作確認
- 運用開始

## 複数コース機能

### 複数コース機能の概要

CoursewareHub では、ひとつのインスタンスで複数の「コース」を管理できる複数コース機能を提供しています。LTI連携のカスタムパラメータを使用することで、Moodleの各コースに対応した独立したJupyter Notebook環境を自動的に提供できます。

<!--
```mermaid
flowchart TB
    subgraph "Moodle (Platform)"
        C1[コース1: Python基礎]
        C2[コース2: データ分析]
        C3[コース3: 機械学習]
    end
    
    subgraph "CoursewareHub (Tool)"
        subgraph "Named Servers"
            NS1[python-basic<br/>Named Server]
            NS2[data-analysis<br/>Named Server]
            NS3[machine-learning<br/>Named Server]
        end
        
        subgraph "ディレクトリ構造"
            D1["/home/{user}/<br>python-basic/"]
            D2["/home/{user}/<br>data-analysis/"]
            D3["/home/{user}/<br>machine-learning/"]
        end
    end
    
    C1 -.->|LTI Launch<br/>course_server=<br>python-basic| NS1
    C2 -.->|LTI Launch<br/>course_server=<br>data-analysis| NS2
    C3 -.->|LTI Launch<br/>course_server=<br>machine-learning| NS3
    
    NS1 --- D1
    NS2 --- D2
    NS3 --- D3
    
    style C1 fill:#bbdefb
    style C2 fill:#c8e6c9
    style C3 fill:#ffcdd2
    style NS1 fill:#e3f2fd
    style NS2 fill:#e8f5e8
    style NS3 fill:#ffebee
```
-->

![複数コース機能](images/cw-411-02.png)

**対象ユーザ:**
- **講師**: 
  - Moodle教師ロール（コース管理権限）を有する
  - CoursewareHubの管理者権限（教材作成・配布のため）を有する
- **受講者**: Moodleにログインした状態で、CoursewareHub への LTI 連携リンクにアクセスして演習を実行

**複数コース機能の特徴:**
- **コース毎の独立環境**: 各コースで独立したNotebook環境とディレクトリを自動作成
- **コンテナイメージ**: コース毎に適切なJupyterコンテナイメージを選択可能
- **データ分離**: コース間でのデータ分離により、セキュアな学習環境を実現

### アーキテクチャ

複数コース機能は、JupyterHubの[Named Server](https://jupyterhub.readthedocs.io/en/latest/howto/configuration/config-user-env.html#named-servers)を活用して実現されています。

**Named Server による実現:**
- **コース毎のコンテンツ**: コースの名前がついたディレクトリによって分けられる
- **ユーザホームディレクトリ**: 各ユーザのホームディレクトリには、コース名でディレクトリが作成される
- **専用演習環境**: コース毎にNamed Serverが作られ、それが専用の演習環境となる
- **受講者に対する制限**: Named Serverの標準機能を利用しつつ、受講者が新規のコースを作成できないように制限

**コース作成プロセス:**
1. **講師による作成**: 講師ユーザが「コース指定付きLTIリンク」を経由してNamed Server を起動
2. **自動ディレクトリ作成**: 以下のディレクトリが自動作成される
   - `/home/{username}/{coursename}/` - 講師の作業用ディレクトリ
   - `/jupyter/admin/info/{coursename}/` - 教材配布用ディレクトリ
   - `/jupyter/admin/textbook/{coursename}/` - テキストブック用ディレクトリ
3. **Named Server起動**: 指定されたコンテナイメージでサーバーが起動
4. **環境変数設定**: `CWH_COURSE_NAME={coursename}` が設定される

**受講者の制限**:
   - 受講者が存在しないコース名で Named Server を起動しようとした場合、エラーメッセージが表示され起動できない

**受講者用ディレクトリ作成**:
   - 受講者が「コース指定付きLTIリンク」を経由して初回ログインした際に、ユーザのHome配下にコース用のディレクトリが自動作成される

### LTIカスタムパラメータ

複数コース機能を利用するためにはMoodleのLTI外部ツール設定で以下のカスタムパラメータを指定します。

#### course_server
- **用途**: コースのNotebookサーバー(Named Server)の名称
- **使用可能な記号**: `-(ハイフン)`、`_（アンダースコア）`、`.（ドット）`、括弧類、`@（アットマーク）`
- **ディレクトリ名**: course_server から、一部のパスに含まれていると不都合な記号を `_`（アンダースコア）に置換した文字列となる

**命名規則の推奨事項:**
- **文字数**: 3～50文字を推奨
- **避けるべき文字**: スペース、日本語文字、特殊記号（`#`, `%`, `&` 等）
- **重複回避**: 既存のコース名と重複しないよう事前確認が必要

**例:**
- 推奨: `data-analysis-2025`, `ml-advanced-spring`
- 非推奨: `データ分析 2025`, `course#1`, `very-long-course-name-that-exceeds-reasonable-length`

#### course_image
- **用途**: Notebookサーバーのコンテナイメージ名（省略時はデフォルトのイメージ）
- **フォーマット**: `"{Name}:{Reference}"`
  - **Name**: CoursewareHubに登録済みの講義用コンテナイメージ名（例: `seaborn`, `network-command`, `python-basic`）
  - **Reference**: イメージのタグやバージョン（例: `head`, `v1.0`, Gitコミットハッシュ）  
    ※大文字は英小文字に変換して指定する
- **有効条件**: course_server パラメータを指定したときのみ有効
- **イメージ登録**:
  - 講義用コンテナイメージはCoursewareHubの管理画面から事前に登録が必要
  - 詳細は「731-講義用Notebook環境のイメージ登録.ipynb」を参照

#### logout-redirect-url
- **用途**: CoursewareHubからログアウトした時の戻り先となるMoodleのアドレス

**設定例:**
```
course_server=python-basic
course_image=python-basic:head
logout-redirect-url=https://moodle.example.org/course/view.php?id=123
```

**結果:**
- Named Server 名: `python-basic`
- コースディレクトリ名: `python-basic`
- 使用イメージ: `python-basic:head`（CoursewareHubに登録済みのコンテナイメージ）
- 環境変数: `CWH_COURSE_NAME=python-basic`

### ディレクトリ構成とセキュリティ

#### 講師用環境
- **ホームディレクトリ**: `/home/{username}/{coursename}` - コース専用の作業領域
- **教材配布用**: `/jupyter/admin/info/{coursename}`, `/jupyter/admin/textbook/{coursename}`

#### 受講者用環境  
- **ホームディレクトリ**: `/home/{username}/{coursename}/` - コース専用の作業領域
- **教材アクセス**: `/home/{username}/{coursename}/info/`, `/home/{username}/{coursename}/textbook/`

#### セキュリティとアクセス制御

**ファイルシステムレベルの分離:**
- **受講者間分離**: 各受講者のホームディレクトリ（`/home/{username}/`）は他の受講者からアクセス不可
- **コース間分離**: コース毎のコンテンツはディレクトリ（`/home/{username}/{coursename}/`）で分離

**Jupyter UIでの制限:**
- **ファイルリスト制限**: 特定コースでログイン時、Jupyterのファイルブラウザでは該当コースのディレクトリのみ表示
- **Named Server制限**: 受講者は管理者が作成したNamed Serverにのみアクセス可能

**Terminal経由でのアクセス:**
- Terminalを使用した場合、他のコースディレクトリへのアクセスが可能

### 運用上の要点

#### 管理者の作業分担
- **Moodle管理者（サイトレベル）**: 外部ツールの登録・管理
- **講師（Moodle教師ロール）**: 「コース指定付きLTIリンク」の作成・管理、受講者管理
- **CoursewareHub管理者**: 教材作成・配布、コンテナイメージ登録

#### コース識別と環境変数
- **CWH_COURSE_NAME**
  - 各notebookでコース識別に使用する環境変数
  - `course_server`パラメータの値がそのまま設定される

## 事前準備

### システム要件の確認

LTI連携を実現するための基本的なシステム要件を確認します。

**CoursewareHub:**
- [23.11.0-20250402](https://github.com/nii-gakunin-cloud/ocs-templates/releases/tag/release%2F23.11.0-20250402)以降のアプリケーションテンプレートでCoursewareHubを構築していること
- auth-proxyサービスが正常に動作していること

**Moodle:**
- LTI 1.3 に対応したMoodle（バージョン 4.1 以降推奨）
- LTI外部ツール機能が有効であること

**動作確認済み環境:**
- Moodle: 4.5.6, 4.1.20

### 権限要件

LTI連携の設定と運用に必要な権限を整理します。

#### Moodleの権限要件

**サイトレベル管理者権限:**
- **必要な作業**: LTI外部ツールのサイト全体登録
- **具体的な権限**: 「サイト管理」→「プラグイン」→「活動モジュール」→「LTI外部ツール」の管理権限
- **作業内容**: CoursewareHubをLTI外部ツールとして登録、プラットフォームID・クライアントID・デプロイメントIDの取得

**講師（Moodle教師ロール）:**
- **必要な作業**: 各コースへのLTIリンク作成
- **具体的な権限**: コースの編集権限、活動・リソースの追加権限
- **作業内容**: コースページへのLTIリンク追加、複数コース機能用カスタムパラメータ設定

#### CoursewareHubの権限要件

**システム管理者権限:**
- **必要な作業**: LTI連携設定、サービス設定
- **具体的な権限**: Ansible実行権限、設定ファイル更新権限、Dockerサービス管理権限
- **作業内容**: LTI設定ファイルの配置、auth-proxyサービスの更新

**CoursewareHub管理者権限（教材作成用）:**
- **必要な作業**: 教材作成・配布、コース環境管理
- **具体的な権限**: JupyterHubのadmin権限、Named Server作成権限
- **付与対象**: Moodle教師ロールを持つ講師

### ネットワーク要件

CoursewareHubとMoodle間のネットワーク接続要件を確認します。

**アクセス要件:**
- CoursewareHubからMoodleへのアクセスが可能であること

### 要件の確認

設定対象のCoursewareHubが要件を満たしていることを確認します。

#### 対象となるCoursewareHub

LTI連携の設定対象となるCoursewareHubのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にして、設定対象となるCoursewareHubのUnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

指定したUnitGroupのVCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

#### システム要件の確認

以下の要件を確認します。

- auth-proxyサービスが正常に動作していること
- 構築されているCoursewareHubが複数コース機能をサポートしていること([23.11.0-20250402](https://github.com/nii-gakunin-cloud/ocs-templates/releases/tag/release%2F23.11.0-20250402)以降のアプリケーションテンプレートが対応）


auth-proxyサービスが正常に動作していることを確認します。

In [ ]:
!ansible {target_hub} -m shell -a '[ $(docker ps -q \
    --filter health=healthy \
    --filter "label=com.docker.swarm.service.name=coursewarehub_auth-proxy" \
    | wc -l) -eq 1 ]'

auth-proxyサービスが複数コースに対応していることを確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'docker exec -it \
    $(docker ps --filter "label=com.docker.swarm.service.name=coursewarehub_auth-proxy" -q) \
    grep -q course_server /var/www/htdocs/php/lti/service.php'

#### ネットワーク要件の確認

以下の要件を確認します。

- CoursewareHubからMoodleへのアクセスが可能であること

LTI連携設定を行うMoodleのURLを指定してください。

In [ ]:
# (例)
# moodle_url = 'https://moodle.example.org'

moodle_url = 

CoursewareHub(auth-proxyサービス)からMoodleへのアクセスが可能であることを確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'docker exec -it \
    $(docker ps --filter "label=com.docker.swarm.service.name=coursewarehub_auth-proxy" -q) \
    curl -vfs -o /dev/null --connect-timeout 10 {moodle_url}'

アクセスできない場合はファイアウォールの設定を変更するなどの対応を行なってください。

##### auth-proxyの/etc/hostsにエントリを追加する

以下の条件でMoodleにアクセスできない場合に、このセクションを実行してください：

**実行が必要な条件:**
- CoursewareHubからMoodleへのDNS解決に失敗する場合
- MoodleがプライベートIPアドレスや内部ホスト名を使用している場合  
- ネットワーク設定でDNSアクセスが制限されている場合

**判断方法:**
前のセルのcurlコマンドで「name resolution failed」や「could not resolve host」エラーが発生した場合

**設定例:**
- 内部サーバー: `("moodle.internal.example.org", "192.168.1.100")`
- 開発環境: `("moodle-dev.local", "10.0.0.50")`

`/etc/hosts`に追加するエントリを次のセルで指定してください。それぞれのエントリは、ホスト名、IPアドレスのタプルを指定してください。

In [ ]:
auth_proxy_extra_hosts = [
    # (例)
    # ("moodle.example.org", "192.168.1.100"),
]

指定された値をgroup_varsに保存します。

In [ ]:
%run scripts/group.py
if auth_proxy_extra_hosts:
    update_group_vars(ugroup_name, auth_proxy_extra_hosts=auth_proxy_extra_hosts)

`docker-compose.yml`のauth-proxyにextra_hostsのエントリを追加するansibleのplaybookを実行します。まずはチェックモードで実行します。

In [ ]:
!ansible-playbook -CDv -l {ugroup_name}_manager playbooks/update-extra-hosts.yml

実際に`docker-compose.yml`の更新を行い、auth-proxyコンテナを起動し直します。

In [ ]:
!ansible-playbook -v -l {ugroup_name}_manager playbooks/update-extra-hosts.yml

Moodleへのアクセスが可能であることを確認します。

In [ ]:
!ansible {target_hub} -m shell -a 'docker exec -it \
    $(docker ps --filter "label=com.docker.swarm.service.name=coursewarehub_auth-proxy" -q) \
    curl -vfs -o /dev/null --connect-timeout 10 {moodle_url}'

## Moodleの設定

CoursewareHubをLTI外部ツールとしてMoodleに登録し、CoursewareHubの設定に必要なパラメータを取得します。この作業にはMoodleのサイトレベル管理者権限が必要です。

### LTI外部ツール登録

Moodleのサイト管理画面からCoursewareHubを外部ツールとして登録します。

#### 管理画面へのアクセス

1. **Moodle管理者権限でログイン**
2. **サイト管理メニューへアクセス**
   - 画面右上の管理メニューから「サイト管理」を選択
3. **プラグイン管理画面への移動**
   - 「プラグイン」→「活動モジュール」→「LTI外部ツール」→「ツールを管理する」を選択

![LTI外部ツール管理画面](images/cw-411-03.png)

#### 新規ツールの追加

1. **ツール追加開始**
   - 「ツールを手動設定」をクリック

![ツール設定開始](images/cw-411-04.png)

2. **基本設定の入力**

![基本設定フォーム](images/cw-411-05.png)

「外部ツール設定」画面でLTI連携に関するCoursewareHubの情報を入力します。次のセルを実行すると、それぞれの入力欄に指定する値が表示されます。

In [ ]:
%run scripts/group.py
from IPython.display import Markdown, display

gvars = load_group_vars(ugroup_name)
cwh_url = f"https://{gvars['master_fqdn']}"
display(Markdown(f"""
| パラメータ | 値 | 備考 |
|:--|:--|:--|
|ツール名||任意の識別しやすい名前<br>(例)`CoursewareHub`|
|ツールURL|`{cwh_url}`|CoursewareHubのURL|
|LTIバージョン|LTI 1.3|LTIのバージョン<br>「LTI 1.3」を指定する|
|公開鍵タイプ|鍵セットURL|ツール認証の方式<br>「鍵セットURL」を指定する|
|公開鍵セット||空欄のままにする|
|ログイン開始URL|`{cwh_url}/php/lti/login.php`||
|リダイレクトURI|`{cwh_url}/php/lti/service.php`||
|カスタムパラメータ||空欄のままにする|
|ツール設定使用|「外部ツール追加時に事前設定ツールとして表示する」または<br>「活動チューザまたは事前設定ツールに表示する」|
|デフォルト起動コンテナ|「新しいウィンドウ」または<br>「既存のウィンドウ」|
"""))

#### 詳細設定の構成

3. **プライバシー設定**
   - **ランチャのメールをツールと共有する**: 常に

CoursewareHubとメールの[ユーザマッピング](#ユーザマッピング)はメールアドレスを識別子として用います。そのためツールの設定でメールの共有設定を有効にする必要があります。

![詳細設定フォーム](images/cw-411-06.png)

4. **設定保存**
   - 「変更を保存する」ボタンをクリック

#### LTI外部ツール設定の完了

5. **登録されたツールの確認**
    - 「ツールを管理する」画面にツールが表示される

![ツールを管理する画面8](images/cw-411-07.png)

### 設定パラメータの確認

CoursewareHubの設定に必要なパラメータをMoodleの画面で確認します。

#### ツール設定詳細の確認

1. **設定済みツール一覧の表示**
   - 「サイト管理」→「プラグイン」→「活動モジュール」→「外部ツール」→「ツールを管理する」
2. **設定詳細の表示**
   - 該当ツールの「設定詳細を表示する」アイコンをクリック

![LTIツール](images/cw-411-08.png)

3. ツール設定詳細

![LTI 1.3設定完了](images/cw-411-09.png)

#### パラメータ一覧

「ツール設定詳細」で表示されたパラメータのうち以下の値を確認してください。次章「5. CoursewareHub側の設定」で使用します。

* プラットフォームID
* クライアントID
* デプロイメントID

## CoursewareHubの設定

前章で取得したMoodleのパラメータを使用してCoursewareHubでLTI連携の設定を行います。

### パラメータの指定

Moodleの外部ツールを登録した際に得られたパラメータを指定します。

プラットフォームIDの値を次のセルに入力してください。

In [ ]:
# (例)
# platform_id = 'https://moodle.example.org'

platform_id = 

クライアントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# client_id = 'xxxxxxxxxxxxxxx'

client_id = 

デプロイメントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# deployment_id = '1'

deployment_id = 

#### パラメータの保存

ここまでに指定したパラメータの値をファイルに保存します。

値を保存する前に簡単なチェックを行います。

In [ ]:
%run scripts/group.py
%run scripts/check_params.py
gvars = load_group_vars(ugroup_name)
lti_config = gvars.get('lti_config', {})
if (platform_id := simple_url_normalize(platform_id)) in lti_config:
    msg = f'既に登録されているplatform_idです: {platform_id}'
    raise RuntimeError(msg)

group_varsに保存します。

In [ ]:
lti_config = gvars.get('lti_config', {})
lti_config[platform_id] = {
    'platform_id': platform_id,
    'client_id': client_id,
    'auth_login_url': simple_url_normalize(platform_id, 'mod/lti/auth.php'),
    'auth_token_url': simple_url_normalize(platform_id, 'mod/lti/token.php'),
    'key_set_url': simple_url_normalize(platform_id, 'mod/lti/certs.php'),
    'deployment_id': deployment_id,
}
update_group_vars(ugroup_name, lti_config=lti_config)

### LTI設定ファイルの配置

CoursewareHubでLTI認証連携を行うための設定ファイル`lti.json`を配置します。

LTI関連の設定ファイルを配置、生成するための Ansible の playbook を実行します。ここで実行するplaybookでは以下の処理を行います。

* 指定されたパラメータに対応するLTIファイルを配置する
* RSAの秘密鍵ファイル`private.key`が存在していない場合は、鍵ペアの生成を行う

まず、実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -CDv -l {target_hub} playbooks/setup-lti.yml || true

実際に設定ファイルの配置を行います。

In [ ]:
!ansible-playbook -Dv -l {target_hub} playbooks/setup-lti.yml

配置した設定ファイル`lti.json`の内容を確認します。

In [ ]:
!ansible {target_hub} -a 'cat {{{{jupyterhub_cfg_dir}}}}/lti/lti.json'

### auth-proxyサービスの更新

LTI設定を反映するためにauth-proxyサービスを更新します。

auth-proxyコンテナが`lti.json`を読み込みさせるため、auth-proxyを再起動します。

In [ ]:
!ansible {target_hub} -a 'docker service update --force coursewarehub_auth-proxy'

コンテナの実行状況を確認します。`auth-proxy`コンテナの状態が`Running`であり、かつ新たに起動されていることを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service ps coursewarehub_auth-proxy'

## Moodleコースでの活用

LTI連携設定が完了したので、実際のMoodleコースでCoursewareHubを活用できるようになります。この作業はMoodle教師ロール権限で実行できます。

### コースへのリンク追加

MoodleコースにCoursewareHubへのLTIリンクを追加します。

#### コース編集モードの有効化

1. **対象コースにアクセス**
   - 講師権限でMoodleにログイン
   - CoursewareHubを利用したいコースを選択

2. **編集モードの有効化**
   - 画面右上に表示されている「編集モード」ボタンをクリック

![コース編集モード](images/cw-411-10.png)

#### LTI外部ツールの追加

3. **「活動またはリソースを追加する」の画面に表示**
   - コースのメニュー[さらに]→[LTI外部ツール]を選択し外部ツール画面を表示する
   - CoursewareHubの[活動チューザに表示する]を有効にする

![コース画面-外部ツール](images/cw-411-11.png)

4. **活動の追加**
   - コース内の任意のセクションで「活動またはリソースを追加する」をクリック

![活動追加](images/cw-411-12.png)

5. **LTI外部ツールの選択**
   - CoursewareHubを選択する

![LTI外部ツール選択](images/cw-411-13.png)

#### CoursewareHubツールの設定

6. **基本設定の入力**
   - **活動名**: `Jupyter Notebook環境` （任意の名前）
   - **活動説明**: 受講者向けの説明文を入力
   
![外部ツール-基本設定](images/cw-411-14.png)

7. **カスタムパラメータの入力**
   - [さらに表示する]を選択する
   - カスタムパラメータに入力する値の詳細については次のセクション以降を参照
   - 単一コースだけを利用する場合でもコース名`course_server`の指定が必要

![カスタムパラメータ](images/cw-411-15.png)

8. **設定の保存**
   - 「保存してコースに戻る」をクリック

### 複数コース対応のカスタムパラメータ設定

複数コース機能を利用するためのカスタムパラメータを設定します。

#### 必須パラメータの設定

- **course_server**
   - コース固有の識別子（例: `python-basic-2025`）
   - 使用可能文字: 英数字、ハイフン、アンダースコア
   - 3～50文字を推奨


#### オプションのパラメータ

- **course_image**
   - 講義用コンテナイメージ（例: `python-basic:head`）
   - 省略時はデフォルトイメージを使用
   - フォーマット: `{Name}:{Reference}`
     - **Name**: CoursewareHubに登録済みの講義用コンテナイメージ名（例: `seaborn`, `network-command`, `python-basic`）
     - **Reference**: イメージのタグやバージョン（例: `head`, `v1.0`, Gitコミットハッシュ, Gitブランチ名）  
       ※大文字は英小文字に変換して指定する

- **logout-redirect-url**
    - CoursewareHubからの戻り先URL（例: `https://moodle.example.org/course/view.php?id=123`）

#### 設定例

以下は「Python基礎 2025年度」コースの設定例です：

```
course_server=python-basic-2025
course_image=python-basic:head
logout-redirect-url=https://moodle.example.org/course/view.php?id=123
```

**この設定による結果**：
- Named Server名: `python-basic-2025`
- コースディレクトリ: `/home/{username}/python-basic-2025/`
- 使用イメージ: `python-basic:head`
- 環境変数: `CWH_COURSE_NAME=python-basic-2025`

### 特定Notebookへの直接リンク

特定のJupyter Notebookファイルに直接リンクする方法を説明します。

#### カスタムパラメータの利用

カスタムパラメータにNotebookを指定するパラメータを追加する。

   - **パラメータ名**: `notebook`
   - **値**: 開きたいNotebookファイルのパス
     - **基準ディレクトリ**: `/home/{username}/{coursename}/`
     - **指定例**: `lesson01.ipynb`
     - **実際のパス**: `/home/{username}/{coursename}/lesson01.ipynb`

#### 注意事項

- **ファイルの存在確認**: 指定したNotebookファイルが実際に存在することを確認
  - 受講者自身が`10_AcquireMaterials.ipynb`を実行するなどの方法で、事前に教材のnotebookを取得する必要がある
- **パスの記述**: コース毎に定まる基準ディレクトリからの相対パスを指定する


## 動作確認

LTI連携が実際に動作することを確認します。

### 講師権限での動作確認

講師アカウントでLTI連携の基本動作を確認します。

#### CoursewareHubの権限

Moodleの講師ユーザに対応するCoursewareHubアカウントには管理者権限が必要となります。これはCoursewareHubで教材作成、配布などの作業を行うために必要な要件となります。

Moodleの講師ユーザをCoursewareHubの管理者として追加するには、以下のnotebookを事前に実行してください。

* 721-管理者の追加.ipynb

#### LTI連携の基本動作を確認する

1. **Moodleコースへのアクセス**
   - 講師権限のアカウントでMoodleにログイン
   - 前章で設定したLTIリンクがあるコースにアクセス
   
![MoodleコースでのLTIリンク](images/cw-411-16.png)

2. **LTIリンクのクリック**
   - CoursewareHubのLTIリンクをクリック
   - CoursewareHubの認証処理が行われ、JupyterHubの画面が表示されることを確認
   - コンテナイメージ選択画面が表示されることを確認
   - カスタムパラメータ`course_image`で選択したコンテナイメージが選択されていることを確認
     - `course_image`を指定していない場合はデフォルトとして設定されているコンテナイメージが選択される

![LTIリンククリック](images/cw-411-17.png)

３. **single-userサーバの起動**
   - コンテナイメージ選択画面の[Start]ボタンをクリックする
   - サーバの起動処理中画面が表示されることを確認する

![single-userサーバの起動](images/cw-411-18.png)

4. **JupyterHub画面の表示**
   - CoursewareHubのJupyterHub画面が正常に表示されることを確認
   - 教師用の管理機能が表示されることを確認:
     - ファイル一覧に`admin_tools/`ディレクトリが表示されること
![教師用JupyterHub画面](images/cw-411-19.png)
     - [Control Panel]を選択した画面に管理者向けの[Admin], [Environments]タブが表示されること
![教師用JupyterHub管理画面](images/cw-411-20.png)

5. **複数コース機能の確認**
   - 環境変数`CWH_COURSE_NAME`にコース名が設定されていること
   - カレントディレクトリが`/home/{username}/{coursename}/`になっていること

![Named Server作成確認](images/cw-411-21.png)

**確認ポイント**:

- LTIリンクが正常にクリック可能
- 認証処理がエラーなく完了
- JupyterHub画面が適切に表示
- 講師用の管理機能にアクセス可能
- Named Serverが正しい名前で作成

### 受講者権限での動作確認

一般的な受講者アカウントでの動作を確認します。

1. **受講者アカウントでのアクセス**
   - 受講者権限のアカウントでMoodleにログイン
   - コースにアクセスしてLTIリンクをクリック

2. **受講者用JupyterHub画面の確認**
   - 受講者用のJupyterHub画面が表示されることを確認
     - コンテナイメージ選択画面が表示されることなく、指定したコンテナイメージでsingle-userサーバが起動する
   - Admin機能にはアクセスできないことを確認
      - ファイル一覧に`admin_tools/`ディレクトリが表示されないこと
      - ファイル一覧に`tools/`ディレクトリが表示されること
![受講者用JupyterHub画面](images/cw-411-22.png)
     - [Control Panel]を選択した画面に管理者向けの[Admin], [Environments]タブが表示されないこと
![受講者用JupyterHub管理画面](images/cw-411-23.png)

**確認ポイント:**
- 受講者アカウントで正常にアクセス可能
- 受講者用画面が適切に表示
- Admin機能へのアクセスが制限


### トラブルシューティング

#### 認証エラー

LTIリンクを選択した時にJupyterHub画面が表示されず、以下のようなエラー画面が表示されることがあります。

![認証エラー](images/cw-411-24.png)

CoursewareHubからMoodleへのアクセスができない場合にこのエラーとなります。[3.4.3 ネットワーク要件の確認](#ネットワーク要件の確認)で行なっているチェックなどを再確認してください。

#### Named Serverへのアクセスがエラーになる

`course_server`を指定したLTIリンクを選択した時に、以下のようなエラー画面が表示されることがあります。

![認可エラー](images/cw-411-25.png)

LTIリンクを選択したユーザ権限により異なる対応が必要となります。

* 講師の場合
  * 原因: Coursewareの管理者権限が設定されていない
  * 対応方法: 「721-管理者の追加.ipynb」を実行して、Moodle教師ロールユーザにCoursewareHubの管理者権限を付与する
* 受講者の場合
  * 原因: 講師が`course_server`のNamed Serverを作成していない
  * 対応方法:
    * 講師が`course_server`のNamed Serverを作成する
    * `course_server`の設定値を修正する

#### Notebookへの直接リンクのアクセスがエラーになる

カスタムパラメータ`notebook`を指定したLTIリンクを選択した時に、以下のようなエラー画面が表示されることがあります。

![notfoundエラー](images/cw-411-26.png)

リンク先のnotebookが存在していない場合、上記のエラーとなります。

notebookは、事前に受講者自身が適切に配置する必要があります。受講者が`tools/10_AcquireMaterials.ipynb`を実行するように指導してください。